# Blockchain Fraud Detection: Data Exploration

This notebook explores the Elliptic dataset, which contains Bitcoin transactions labeled as licit or illicit. We'll analyze the data structure, feature distributions, and network characteristics.

## Dataset Overview

The Elliptic dataset consists of:
- **nodes.csv**: Bitcoin transactions with features and labels
- **edges.csv**: Transaction flows connecting the nodes

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.decomposition import PCA
import warnings

# Set plotting style
sns.set(style="whitegrid")
plt.style.use('seaborn-v0_8-whitegrid')
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)

## 1. Load and Inspect Data

In [ ]:
# Load data
nodes_path = '../data/raw/nodes.csv'
edges_path = '../data/raw/edges.csv'

df_nodes = pd.read_csv(nodes_path)
df_edges = pd.read_csv(edges_path)

print(f"Node data shape: {df_nodes.shape}")
print(f"Edge data shape: {df_edges.shape}")

In [ ]:
# Examine node data
print("Node data sample:")
df_nodes.head()

In [ ]:
# Examine edge data
print("Edge data sample:")
df_edges.head()

In [ ]:
# Check for missing values
print("Missing values in node data:")
print(df_nodes.isnull().sum().sum())

print("\nMissing values in edge data:")
print(df_edges.isnull().sum().sum())

## 2. Analyze Target Distribution (Licit vs. Illicit)

In [ ]:
# Get target distribution
target_counts = df_nodes['class'].value_counts()
print("Target distribution:")
print(target_counts)
print(f"Fraud percentage: {target_counts[1] / len(df_nodes) * 100:.2f}%")

In [ ]:
# Visualize target distribution
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='class', data=df_nodes, palette=['#4285F4', '#EA4335'])
plt.title('Transaction Class Distribution', fontsize=15)
plt.xlabel('Class (0=Legitimate, 1=Fraudulent)', fontsize=12)
plt.ylabel('Count', fontsize=12)

# Add count labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():,}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()

# Pie chart
plt.figure(figsize=(8, 6))
plt.pie(target_counts, labels=['Legitimate', 'Fraudulent'], 
        autopct='%1.2f%%', startangle=90, colors=['#4285F4', '#EA4335'],
        wedgeprops={'edgecolor': 'w', 'linewidth': 1.5}, explode=[0.02, 0.05])
plt.title('Transaction Class Distribution', fontsize=15)
plt.axis('equal')
plt.tight_layout()
plt.show()

## 3. Explore Time Steps

In [ ]:
# Check if there's a time step column
if 'time_step' in df_nodes.columns:
    # Analyze transactions per time step
    time_step_counts = df_nodes['time_step'].value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    time_step_counts.plot(kind='bar', color='skyblue')
    plt.title('Transactions per Time Step', fontsize=15)
    plt.xlabel('Time Step', fontsize=12)
    plt.ylabel('Number of Transactions', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Analyze fraud distribution over time
    fraud_by_time = df_nodes.groupby('time_step')['class'].mean()
    
    plt.figure(figsize=(12, 6))
    fraud_by_time.plot(kind='line', marker='o', color='crimson')
    plt.title('Fraud Percentage Over Time', fontsize=15)
    plt.xlabel('Time Step', fontsize=12)
    plt.ylabel('Percentage of Fraudulent Transactions', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No 'time_step' column found in the data")
    # Try to identify if another column might represent time
    for col in df_nodes.columns[2:10]:  # Check first few columns
        unique_vals = df_nodes[col].nunique()
        if 30 < unique_vals < 100:  # Potential time step column
            print(f"Column {col} has {unique_vals} unique values and might represent time steps")

## 4. Feature Analysis

In [ ]:
# Basic statistics of features
feature_stats = df_nodes.iloc[:, 2:].describe().T
feature_stats['variance'] = df_nodes.iloc[:, 2:].var()
feature_stats = feature_stats.sort_values('variance', ascending=False)

print("Top 10 features by variance:")
feature_stats.head(10)

In [ ]:
# Find low-variance features
low_var_threshold = 0.01
low_var_features = feature_stats[feature_stats['variance'] < low_var_threshold].index.tolist()

print(f"Number of low-variance features (variance < {low_var_threshold}): {len(low_var_features)}")
print("Examples of low-variance features:")
feature_stats.loc[low_var_features].head()

In [ ]:
# Correlation with target
feature_correlation = df_nodes.iloc[:, 2:].corrwith(df_nodes['class']).abs().sort_values(ascending=False)

print("Top 15 features by correlation with fraud label:")
feature_correlation.head(15)

In [ ]:
# Visualize top feature correlations
plt.figure(figsize=(12, 6))
feature_correlation.head(15).plot(kind='bar', color='teal')
plt.title('Top 15 Features by Correlation with Fraud Label', fontsize=15)
plt.xlabel('Feature', fontsize=12)
plt.ylabel('Absolute Correlation', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# PCA to visualize feature space
features = df_nodes.iloc[:, 2:].values
labels = df_nodes['class'].values

# Apply PCA
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features)

# Create DataFrame for plotting
df_pca = pd.DataFrame({
    'PC1': features_pca[:, 0],
    'PC2': features_pca[:, 1],
    'class': labels
})

# Visualize PCA projection
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='class', palette=['#4285F4', '#EA4335'], 
                alpha=0.6, s=50)
plt.title('PCA Projection of Transaction Features', fontsize=15)
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)', fontsize=12)
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)', fontsize=12)
plt.legend(title='Class', labels=['Legitimate', 'Fraudulent'])
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Graph Structure Analysis

In [ ]:
# Create graph from edges
G = nx.DiGraph()

# Add edges
for _, row in df_edges.iterrows():
    G.add_edge(row[0], row[1])

print(f"Graph statistics:")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Is directed: {nx.is_directed(G)}")
print(f"Is connected: {nx.is_weakly_connected(G)}")

In [ ]:
# Create node mapping for faster lookups
node_id_to_class = {row[0]: row[1] for _, row in df_nodes.iterrows()}

# Analyze node degrees
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())
total_degrees = dict(G.degree())

# Create a DataFrame for analysis
df_degrees = pd.DataFrame({
    'node_id': list(G.nodes()),
    'in_degree': [in_degrees.get(n, 0) for n in G.nodes()],
    'out_degree': [out_degrees.get(n, 0) for n in G.nodes()],
    'total_degree': [total_degrees.get(n, 0) for n in G.nodes()],
})

# Add class information where available
df_degrees['class'] = df_degrees['node_id'].map(lambda x: node_id_to_class.get(x, -1))
df_degrees = df_degrees[df_degrees['class'] != -1]  # Keep only nodes with class information

print("Degree statistics:")
print(df_degrees[['in_degree', 'out_degree', 'total_degree']].describe())

In [ ]:
# Compare degrees by class
df_degrees_by_class = df_degrees.groupby('class')[['in_degree', 'out_degree', 'total_degree']].mean()
print("Average degrees by class:")
print(df_degrees_by_class)

In [ ]:
# Visualize degree distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# In-degree distribution
sns.histplot(data=df_degrees, x='in_degree', hue='class', 
             kde=True, bins=30, palette=['#4285F4', '#EA4335'], ax=axes[0])
axes[0].set_title('In-Degree Distribution by Class', fontsize=14)
axes[0].set_xlabel('In-Degree', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].legend(title='Class', labels=['Legitimate', 'Fraudulent'])

# Out-degree distribution
sns.histplot(data=df_degrees, x='out_degree', hue='class', 
             kde=True, bins=30, palette=['#4285F4', '#EA4335'], ax=axes[1])
axes[1].set_title('Out-Degree Distribution by Class', fontsize=14)
axes[1].set_xlabel('Out-Degree', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].legend(title='Class', labels=['Legitimate', 'Fraudulent'])

# Total degree distribution
sns.histplot(data=df_degrees, x='total_degree', hue='class', 
             kde=True, bins=30, palette=['#4285F4', '#EA4335'], ax=axes[2])
axes[2].set_title('Total Degree Distribution by Class', fontsize=14)
axes[2].set_xlabel('Total Degree', fontsize=12)
axes[2].set_ylabel('Count', fontsize=12)
axes[2].legend(title='Class', labels=['Legitimate', 'Fraudulent'])

plt.tight_layout()
plt.show()

In [ ]:
# Analyze the strongly connected components
strongly_connected = list(nx.strongly_connected_components(G))
weakly_connected = list(nx.weakly_connected_components(G))

print(f"Number of strongly connected components: {len(strongly_connected)}")
print(f"Number of weakly connected components: {len(weakly_connected)}")

# Size of largest components
sizes_strongly = [len(c) for c in strongly_connected]
sizes_weakly = [len(c) for c in weakly_connected]

print(f"Size of largest strongly connected component: {max(sizes_strongly)}")
print(f"Size of largest weakly connected component: {max(sizes_weakly)}")

In [ ]:
# Visualize component size distribution
plt.figure(figsize=(12, 6))

# Filter out tiny components for better visualization
min_size = 5
filtered_sizes = [s for s in sizes_strongly if s >= min_size]

plt.hist(filtered_sizes, bins=30, alpha=0.7, color='purple')
plt.title(f'Size Distribution of Strongly Connected Components (size >= {min_size})', fontsize=15)
plt.xlabel('Component Size', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize a small subgraph for illustration
# Choose the largest connected component that's not too large
for component in sorted(strongly_connected, key=len, reverse=True):
    if 50 <= len(component) <= 100:
        sample_component = component
        break
else:
    # If no suitable component found, take a random subset of nodes
    sample_nodes = list(G.nodes())[:100]
    sample_component = sample_nodes

sample_subgraph = G.subgraph(sample_component)

# Color nodes by class
node_colors = []
for node in sample_subgraph.nodes():
    if node in node_id_to_class:
        if node_id_to_class[node] == 1:
            node_colors.append('#EA4335')  # Red for fraudulent
        else:
            node_colors.append('#4285F4')  # Blue for legitimate
    else:
        node_colors.append('gray')  # Gray for unknown

plt.figure(figsize=(12, 10))
pos = nx.spring_layout(sample_subgraph, seed=42)  # Position nodes using force-directed layout

nx.draw_networkx_nodes(sample_subgraph, pos, node_color=node_colors, node_size=100, alpha=0.8)
nx.draw_networkx_edges(sample_subgraph, pos, alpha=0.2, arrowsize=10)

plt.title('Sample Transaction Subgraph', fontsize=16)
plt.axis('off')

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#4285F4', markersize=10, label='Legitimate'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#EA4335', markersize=10, label='Fraudulent'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', markersize=10, label='Unknown')
]
plt.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

## 6. Analyze Fraud Network Patterns

In [ ]:
# Identify fraud nodes
fraud_nodes = df_nodes[df_nodes['class'] == 1]['txId'].tolist()
legitimate_nodes = df_nodes[df_nodes['class'] == 0]['txId'].tolist()

# Calculate fraud connections
fraud_to_fraud = 0
fraud_to_legitimate = 0
legitimate_to_fraud = 0
legitimate_to_legitimate = 0

for _, row in df_edges.iterrows():
    source, target = row[0], row[1]
    
    if source in fraud_nodes and target in fraud_nodes:
        fraud_to_fraud += 1
    elif source in fraud_nodes and target in legitimate_nodes:
        fraud_to_legitimate += 1
    elif source in legitimate_nodes and target in fraud_nodes:
        legitimate_to_fraud += 1
    elif source in legitimate_nodes and target in legitimate_nodes:
        legitimate_to_legitimate += 1

print("Transaction flow patterns:")
print(f"Fraud → Fraud: {fraud_to_fraud}")
print(f"Fraud → Legitimate: {fraud_to_legitimate}")
print(f"Legitimate → Fraud: {legitimate_to_fraud}")
print(f"Legitimate → Legitimate: {legitimate_to_legitimate}")

In [ ]:
# Visualize transaction flow patterns
flow_labels = ['Fraud → Fraud', 'Fraud → Legitimate', 'Legitimate → Fraud', 'Legitimate → Legitimate']
flow_values = [fraud_to_fraud, fraud_to_legitimate, legitimate_to_fraud, legitimate_to_legitimate]

plt.figure(figsize=(10, 6))
bars = plt.bar(flow_labels, flow_values, color=['#EA4335', '#F4B400', '#F4B400', '#4285F4'])

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{height:,}', ha='center', va='bottom', fontsize=10)

plt.title('Transaction Flow Patterns', fontsize=15)
plt.ylabel('Number of Transactions', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Create a log scale plot for better visualization of all categories
plt.figure(figsize=(10, 6))
bars = plt.bar(flow_labels, flow_values, color=['#EA4335', '#F4B400', '#F4B400', '#4285F4'], log=True)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height * 1.1,
             f'{height:,}', ha='center', va='bottom', fontsize=10)

plt.title('Transaction Flow Patterns (Log Scale)', fontsize=15)
plt.ylabel('Number of Transactions (Log Scale)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Summary of Findings

From our exploration of the Elliptic dataset, we can draw the following conclusions:

1. **Data Overview**: The dataset contains [X] transactions (nodes) and [Y] transaction flows (edges).

2. **Class Distribution**: Around [Z]% of transactions are labeled as fraudulent, making this an imbalanced classification problem.

3. **Feature Analysis**:
   - [X] features have very low variance and might be removed.
   - Several features show strong correlation with the fraud label.
   - PCA visualization shows some separation between legitimate and fraudulent transactions.

4. **Graph Structure**:
   - The transaction network is directed and [weakly/strongly] connected.
   - It consists of multiple connected components.
   - Fraudulent transactions tend to have [higher/lower] degrees compared to legitimate ones.

5. **Fraud Patterns**:
   - Most transaction flows occur between legitimate nodes.
   - A significant number of legitimate-to-fraud and fraud-to-legitimate transactions exist.
   - Fraud-to-fraud transactions are relatively rare.

These insights will inform our feature engineering and model development approaches in the next notebooks.